In [ ]:
#pip install git+https://github.com/LIAAD/yake
#pip install keybert
#pip install rake-nltk

In [ ]:

import pandas as pd
import pathlib


class DataHandler:
    def __init__(self, location):
        self.type = pathlib.Path(location).suffix
        self.dataset_location = location
        
#         Use the pandas dataframe to get columns 
        if self.type == '.csv':
            self.df = pd.read_csv(location, nrows=5000)
        elif self.type == '.xlsx':
            self.df = pd.read_excel(location, nrows=5000)

    def get_dataframe(self):
        return self.df

In [ ]:
import xlsxwriter
def store(name, extracted_keywords):
    workbook = xlsxwriter.Workbook(name+'.xlsx')
    worksheet = workbook.add_worksheet()
    def convert_tuple(value):
        if not isinstance(value, tuple):
            return value
        return str(value[0])
    col = 0
    for row, data in enumerate(extracted_keywords):
        data = map(convert_tuple, data)
        worksheet.write_row(row, col, data)

    workbook.close()

In [ ]:
import yake
def yakealgo():
    language = "en"
    max_ngram_size = 3
    deduplication_thresold = 0.9
    deduplication_algo = 'seqm'
    windowSize = 1
    numOfKeywords = 20
    kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_thresold, dedupFunc=deduplication_algo, windowsSize=windowSize, top=numOfKeywords, features=None)
    dh = DataHandler("Data.xlsx")
    df = dh.get_dataframe()
    extracted_keywords = []
    for abs in df['Abstract']:
        keywords = kw_extractor.extract_keywords(abs)
        extracted_keywords.append(keywords)
    store("yake", extracted_keywords)

In [ ]:
from keybert import KeyBERT
def keybertalgo():
    model = KeyBERT('distilbert-base-nli-mean-tokens')

    dh = DataHandler("Data.xlsx")
    df = dh.get_dataframe()
    extracted_keywords = []
    for abs in df['Abstract']:
        keywords = model.extract_keywords(abs, keyphrase_ngram_range=(1, 3), stop_words=None,use_maxsum=True, nr_candidates=20, top_n=20)
        extracted_keywords.append(keywords)
    store("keybert", extracted_keywords)

In [ ]:
from rake_nltk import Metric, Rake
def rakealgo():
    dh = DataHandler("Data.xlsx")
    df = dh.get_dataframe()
    extracted_keywords = []
    for abs in df['Abstract']:
        r = Rake(min_length=1, max_length=3)
        r.extract_keywords_from_text(abs)
        #r.get_ranked_phrases()
        #r.get_ranked_phrases_with_scores()
        result = r.get_ranked_phrases()
        topResult = result[:20]
        extracted_keywords.append(topResult)
    store("rake", extracted_keywords)

In [ ]:
yakealgo()
keybertalgo()
rakealgo()